In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import glob

In [2]:
weather = pd.DataFrame()
weather_features = ['humidity', 'precipitation', 'temperature', 'weather_observation', 'wind_direction', 'wind_speed']
for i in weather_features:

    filepaths = glob.glob(f"main-project/final(sorted)/{i}/*.csv")
    print(filepaths)
    dfs = []

    for path in filepaths:
        df = pd.read_csv(path)
        dfs.append(df)

    weather_ = dfs[0]
    for df in dfs[1:]:
        weather_ = pd.concat([df, weather_])

    if len(weather) == 0:
        weather = weather_
    else: 
        weather_.drop(columns=['station_id'], inplace=True)
        weather = weather_.merge(weather, on=['date', 'time'], how='outer', suffixes=('', '_drop'))
    # weather = weather.loc[:, ~weather_.columns.str.contains("_drop$")]
weather

['main-project/final(sorted)/humidity\\HN_humidity2023.csv', 'main-project/final(sorted)/humidity\\HN_humidity2024.csv', 'main-project/final(sorted)/humidity\\HN_humidity_2019.csv', 'main-project/final(sorted)/humidity\\HN_humidity_2020.csv', 'main-project/final(sorted)/humidity\\HN_humidity_2021.csv', 'main-project/final(sorted)/humidity\\HN_humidity_2022.csv']
['main-project/final(sorted)/precipitation\\HN_precipitation_2019_2021.csv', 'main-project/final(sorted)/precipitation\\HN_precipitation_2024.csv', 'main-project/final(sorted)/precipitation\\HN_precipitation_total_12h2022.csv', 'main-project/final(sorted)/precipitation\\HN_precipitation_total_12h_2023.csv']
['main-project/final(sorted)/temperature\\HN_temperature2023.csv', 'main-project/final(sorted)/temperature\\HN_temperature2024.csv', 'main-project/final(sorted)/temperature\\HN_temperature_2019.csv', 'main-project/final(sorted)/temperature\\HN_temperature_2020.csv', 'main-project/final(sorted)/temperature\\HN_temperature_202

,date,time,wind speed,wind direction,weather observation,temperature,precipitation,station_id,humidity
0,20240101,12:00am,13km/h,east-southeast (110°),NaN,22.0°C,NaN,488200.0,83%
1,20240101,01:00am,13km/h,southeast (130°),NaN,22.0°C,NaN,488200.0,88%
2,20240101,02:00am,9km/h,southeast (130°),NaN,22.0°C,NaN,488200.0,88%
3,20240101,03:00am,11km/h,east-southeast (110°),NaN,21.0°C,NaN,488200.0,94%
4,20240101,04:00am,6km/h,southeast (130°),NaN,21.0°C,NaN,488200.0,94%
...,...,...,...,...,...,...,...,...,...
51615,20220611,10:30pm,NaN,NaN,mist,NaN,NaN,NaN,NaN
51616,20210404,12:00pm,NaN,NaN,NaN,29.0°C,NaN,488200.0,74%
51617,20200424,11:00am,NaN,NaN,NaN,18.0°C,NaN,488200.0,77%
51618,20220501,07:00am,NaN,NaN,NaN,NaN,12.0mm,NaN,NaN


In [3]:
format_weather = weather.copy()
#format the datetime columns
format_weather['date_format'] = format_weather['date'].apply(lambda x: str(x)[:4]+"-"+ str(x)[4:6]+"-"+ str(x)[6:])
format_weather['date_format'] = pd.to_datetime(format_weather['date_format'])

format_weather['time_format'] = pd.to_datetime(format_weather['time'])
# format_weather['datetime_format'] = pd.to_datetime(format_weather['date_format'] + ' ' + format_weather['time'])
format_weather['datetime'] = format_weather.apply(lambda x: x['time_format'].replace(year=x['date_format'].year,
                                                                                    month=x['date_format'].month,
                                                                                    day=x['date_format'].day), axis=1)

#format temperature column
format_weather['temperature'] = format_weather['temperature'].astype(str).str.strip().str[:-2]
format_weather['temperature'] = pd.to_numeric(format_weather['temperature'], errors='coerce')

daily_avg_temp = format_weather.groupby('date_format')['temperature'].transform('mean')
format_weather['temperature'] = format_weather['temperature'].fillna(daily_avg_temp)
# format_weather['temperature'] = format_weather['temperature'].interpolate(method='time')
format_weather['temperature'] = format_weather['temperature'].ffill().bfill()  # Fill remaining NaNs

#format humidity column
format_weather['humidity'] = format_weather['humidity'].astype(str).str.strip().str[:-1]
format_weather['humidity'] = pd.to_numeric(format_weather['humidity'], errors='coerce')

daily_avg_humid = format_weather.groupby('date_format')['humidity'].transform('mean')
format_weather['humidity'] = format_weather['humidity'].fillna(daily_avg_humid)

#format wind direction
extracted = format_weather['wind direction'].astype(str).str.extract(r'\((\d+)[^\d]*\)')
format_weather['wind direction'] = extracted[0].fillna(format_weather['wind direction'])
format_weather['wind_direction'] = pd.to_numeric(format_weather['wind direction'], errors='ignore')

format_weather['wind_direction'] = format_weather['wind_direction'].ffill()

#format wind speed
format_weather['wind speed'] = format_weather['wind speed'].astype(str).str.strip().str[:-4]
format_weather['wind speed'] = pd.to_numeric(format_weather['wind speed'], errors='coerce')

format_weather['wind speed'] = format_weather['wind speed'].ffill()

#format weather observation

#format precipitation
extracted = format_weather['precipitation'].astype(str).str.extract(r'(-?\d+\.\d+)')
format_weather['precipitation'] = extracted
format_weather['precipitation'] = pd.to_numeric(format_weather['precipitation'], errors='coerce')

format_weather

C:\Users\admin\AppData\Local\Temp\ipykernel_52680\2065082316.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  format_weather['time_format'] = pd.to_datetime(format_weather['time'])


,date,time,wind speed,wind direction,weather observation,temperature,precipitation,station_id,humidity,date_format,time_format,datetime,wind_direction
0,20240101,12:00am,13.0,110,NaN,22.000000,NaN,488200.0,83.000000,2024-01-01,2024-12-16 00:00:00,2024-01-01 00:00:00,110
1,20240101,01:00am,13.0,130,NaN,22.000000,NaN,488200.0,88.000000,2024-01-01,2024-12-16 01:00:00,2024-01-01 01:00:00,130
2,20240101,02:00am,9.0,130,NaN,22.000000,NaN,488200.0,88.000000,2024-01-01,2024-12-16 02:00:00,2024-01-01 02:00:00,130
3,20240101,03:00am,11.0,110,NaN,21.000000,NaN,488200.0,94.000000,2024-01-01,2024-12-16 03:00:00,2024-01-01 03:00:00,110
4,20240101,04:00am,6.0,130,NaN,21.000000,NaN,488200.0,94.000000,2024-01-01,2024-12-16 04:00:00,2024-01-01 04:00:00,130
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51615,20220611,10:30pm,7.0,NaN,mist,28.208333,NaN,NaN,89.208333,2022-06-11,2024-12-16 22:30:00,2022-06-11 22:30:00,320
51616,20210404,12:00pm,7.0,NaN,NaN,29.000000,NaN,488200.0,74.000000,2021-04-04,2024-12-16 12:00:00,2021-04-04 12:00:00,320
51617,20200424,11:00am,7.0,NaN,NaN,18.000000,NaN,488200.0,77.000000,2020-04-24,2024-12-16 11:00:00,2020-04-24 11:00:00,320
51618,20220501,07:00am,7.0,NaN,NaN,22.791667,12.0,NaN,60.208333,2022-05-01,2024-12-16 07:00:00,2022-05-01 07:00:00,320


In [62]:
format_weather['wind_direction'].isna().sum()

0

In [66]:
df = format_weather.groupby('date_format')['weather observation']
df.value_counts()

date_format  weather observation        
2019-01-04   mist                           3
             cloudiness unchanged           1
             continuous light drizzle       1
2019-01-05   mist                           4
             continuous moderate drizzle    1
                                           ..
2024-11-03   mist                           1
2024-11-05   clouds developing              1
2024-11-09   mist                           1
2024-11-11   mist                           1
2024-11-15   mist                           1
Name: count, Length: 3636, dtype: int64

In [56]:
format_weather['precipitation'].value_counts()

precipitation
0.0      275
0.1      127
2.0      122
1.0      104
3.0       81
        ... 
96.0       1
101.0      1
179.0      1
146.0      1
83.0       1
Name: count, Length: 88, dtype: int64

In [4]:
format_weather.sort_values(by='datetime', inplace=True)
format_weather[['datetime', 'temperature', 'humidity', 'precipitation', 'wind_direction', 'wind speed', 'weather observation']].to_csv('weather_data.csv', index=False, mode='w+')